# Исследование надёжности заёмщиков

Кредитный отдел банка заказал исследование - влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

От заказчика предоставлена статистика платежеспособности клиентов с разными характеристиками. Результаты исследования будут учтены при построении модели **кредитного скоринга**



# Цель 

- Выснить влияние семейного положения и количества детей на возврат кредита в срок

- Оценить влияние других факторов на возврат кредита в срок



# Описание данных 

- children — количество детей в семье

- days_employed — общий трудовой стаж в днях

- dob_years — возраст клиента в годах

- education — уровень образования клиента

- education_id — идентификатор уровня образования

- family_status — семейное положение

- family_status_id — идентификатор семейного положения

- gender — пол клиента

- income_type — тип занятости

- debt — имел ли задолженность по возврату кредитов

- total_income — ежемесячный доход

- purpose — цель получения кредита


## Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem 
m = Mystem()
import warnings
warnings.simplefilter('ignore')

try:
    df = pd.read_csv('E:/Python/Project 1/data.csv') 
except:
    df = pd.read_csv('/datasets/data.csv')
df
#df.info()
#df['education'].unique()
#df['income_type'].unique()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод**

Предоставлена таблица данных на более чем 20 тыс. строк. Видно, что в данных много недочетов, такие как: дубликаты, отсутсвующие значения, отрицательные значения и т.д. В общем, данные требуют тщательной предобработки перед тем как приступить к анализу.

## Предобработка данных

### Обработка пропусков

In [2]:
#df.info() 
# Отрабатываем отклонения в столбце дети
df['children'].value_counts() 
df.loc[df['children'] == -1] 
df.loc[df['children'] == 20] 
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2) 
df['children'].value_counts() 

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [3]:
# Отрабатываем отклонения в столбце стаж
df[df['days_employed'].isnull()] 
df['days_employed'] = df['days_employed'].fillna(0) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# Отрабатываем отклонения в столбце возраст
df['dob_years'].value_counts() 
df['dob_years'] = df['dob_years'].replace(0, df['dob_years'].median()) 
df['dob_years'].value_counts() 

42.0    698
35.0    617
40.0    609
41.0    607
34.0    603
38.0    598
33.0    581
39.0    573
31.0    560
36.0    555
44.0    547
29.0    545
30.0    540
48.0    538
37.0    537
50.0    514
43.0    513
32.0    510
49.0    508
28.0    503
45.0    497
27.0    493
56.0    487
52.0    484
47.0    480
54.0    479
46.0    475
58.0    461
57.0    460
53.0    459
51.0    448
59.0    444
55.0    443
26.0    408
60.0    377
25.0    357
61.0    355
62.0    352
63.0    269
64.0    265
24.0    264
23.0    254
65.0    194
66.0    183
22.0    183
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

In [5]:
# Проверка отклонений в столбцах образование, идентификатор образования, семейное положение и идентификатор семейного положения
df['education'].value_counts() 
df['education_id'].value_counts() 
df['family_status'].value_counts() 
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [6]:
# Отрабатываем отклонения в столбце пол
df['gender'].value_counts() 
df.loc[df['gender'] == 'XNA'] 
df['gender'] = df['gender'].replace('XNA', 'F') 
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [7]:
# Проверка отклонений в столбцах тип занятости, наличие задолженности и цель кредита
df['income_type'].value_counts() 
#df['debt'].value_counts() 
#df['purpose'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [8]:
# Отрабатываем отклонения в столбце ежемесячный доход, заполняя пропуски медианным доходом по группам 

df.info() 
df[df['total_income'].isnull()]['income_type'].unique()

all_income_type = ['сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'предприниматель']
all_education_id = [0, 1, 2, 3, 4]

for element in all_income_type:
    for selement in all_education_id:
        med = df[(df['income_type'] == element) & (df['education_id'] == selement)&(df['dob_years'] < 30)]['total_income'].median()
        df.loc[(df['income_type'] == element) & (df['education_id'] == selement)&(df['dob_years'] < 30), 'total_income'] = df.loc[(df['income_type'] == element) & (df['education_id'] == selement)&(df['dob_years'] < 30), 'total_income'].fillna(med)
        med = df[(df['income_type'] == element) & (df['education_id'] == selement)&(df['dob_years'] >= 60)]['total_income'].median()
        df.loc[(df['income_type'] == element) & (df['education_id'] == selement)&(df['dob_years'] >= 60), 'total_income'] = df.loc[(df['income_type'] == element) & (df['education_id'] == selement)&(df['dob_years'] >= 60), 'total_income'].fillna(med)
        med = df[(df['income_type'] == element) & (df['education_id'] == selement)&(df['dob_years'] >= 30)&(df['dob_years'] < 60)]['total_income'].median()
        df.loc[(df['income_type'] == element) & (df['education_id'] == selement)&(df['dob_years'] >= 30)&(df['dob_years'] < 60), 'total_income'] = df.loc[(df['income_type'] == element) & (df['education_id'] == selement)&(df['dob_years'] >= 30)&(df['dob_years'] < 60), 'total_income'].fillna(med)

df.info()        
        
# В двух оставшихся заявках заполним доход вручную
df[df['total_income'].isnull()]       
med = df[(df['income_type'] == 'госслужащий') & (df['education_id'] == 3)]['total_income'].median()
df.loc[8142, 'total_income'] = med           
df[(df['income_type'] == 'предприниматель')] 
df.loc[5936, 'total_income'] =df.loc[18697, 'total_income']
df.info()            

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------    

**Вывод**

По результатам проверки отсутсвующие значения находятся только с столбце стажа и ежемесячного дохода, причем количество пропусков одинаковое.

В столбце дети:
В 47 заявках указано количество детей -1 и в 76 указано 20, эти значения явно ошибочны, отрицательного быть не может, а ближайшее к 20 реальное значение детей 5 и таких всего 9 заявок. Предполагаю, что -1 это на самом деле 1, поэтому произведу замену, тем более 47 зявок это менее 1% от заявок с 1 ребнком, явного влияния не окажет на результат и семейный статус большинства соответсвует детям. По той же логике меняю 20 на 2. Причиной появления данных ошибок может быть человеческий фактор.

В столбце стаж:
Есть отрицательные значения, цифры разного порядка, но в разрезе нашей задачи это нам не мешает. Строки с отсутсвующими значениями совпадают с отсуствующими значениями в столбце ежемесячный доход. Это говорит о том, что по данным заявкам кредиты выдавались без подтверждения дохода и работы. Поэтому заменим на условные 0.

В столбце возраст:
Значений возраста = 0 оказалось 101, это менее 1% от общего количества. Заменяю на медианное значение.

В столбце образование:
Нулевых значений нет, но много дубликатов. Займемся ими чуть позже.

В столбце индикатор образования:
Все в порядке, пропусков нет.

В столбце семейное положение:
Все в порядке, пропусков нет.

В столбце индикатор семейного положения:
Все в порядке, пропусков нет.

В столбце пол:
У одного человека пол не определился. Женщин гораздо больше, поэтому направим его к ним.

В столбце тип занятости:
Все в порядке, пропусков нет, хотя некоторые значения представлены в единичном экземпляре. В разрезе данного задания это значения не имеет, но для более наглядного предоставления данных стоило вынести их в комментарий исследования.

В столбце наличия задолженности:
Все в порядке, пропусков нет.

В столбце ежемесячный доход:
Есть пропуски, заменим их на медианные значения по группам, объединенные по характеристикам тип занятости + образование + возрастная группа.


В столбце цель кредита:
Много смысловых дубликатов, которые можно объеденить в одну категорию. Пропусков и нулевых значений нет.


### Замена типа данных

In [9]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int') 
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Вещественный тип обнаружен в столбцах стаж и ежемесячный доход. Изменяем методом astype() в целочисленные значения.

### Обработка дубликатов

In [10]:
df['education']=df['education'].str.lower()
df['education'].value_counts() 
df.duplicated().sum() 
df = df.drop_duplicates().reset_index(drop=True) 
df.duplicated().sum() 

0

**Вывод**

Посчитано количество дубликатов методом duplicated() и далее удалены, восстановив индексы. Возможной причиной появления данных дубликатов может быть сбой в системе.

Дубликаты встречаются в столбцах образование и цель кредита.

В столбце образование дубликаты из за использования разного регистра. Приведем все к нижнему регистру.

Столбец цель кредита обработаем далее.

### Лемматизация

In [11]:
df['purpose'].value_counts() 

df['purpose_lemma'] = df['purpose'].apply(m.lemmatize)

df['purpose_lemma'].value_counts() 

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       767
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

In [12]:
# Функция разброса по категориям цели кредита
def purpose_category(lem):
    if 'свадьба' in lem:
        return 'свадьба'
    elif ('сдача' in lem) or ('коммерческий' in lem):
        return 'покупка коммерческой недвижимости'
    elif 'строительство' in lem:
        return 'строительство'
    elif 'ремонт' in lem:
        return 'ремонт'
    elif ('жилье' in lem) or ('недвижимость' in lem): 
        return 'покупка жилья'
    elif 'автомобиль' in lem:
        return 'покупка автомобиля'
    elif 'образование' in lem:
        return 'образование'
    return 'другое'

In [13]:
df['purpose_category'] = df['purpose_lemma'].apply(purpose_category) 
df['purpose_category'].value_counts() 

df[df['purpose_category'] == 'покупка жилья']['purpose'].value_counts() 

операции с недвижимостью           675
операции с жильем                  652
покупка жилья                      646
жилье                              646
покупка жилья для семьи            638
недвижимость                       633
операции со своей недвижимостью    627
покупка недвижимости               621
покупка своего жилья               620
покупка жилой недвижимости         606
Name: purpose, dtype: int64

**Вывод**

Проанализировав количество уникальных значений в столбце цель кредита, выведено несколько общих категорий: 
- свадьба
- покупка жилья
- покупка коммерческой недвижимости
- строительство
- ремонт
- покупка автомобиля
- образование 

Для этого создано 2 новых столбца, один с леммами, второй с новыми категориями.

### Категоризация данных

In [14]:
med = df['total_income'].median() 

def class_income(income):
    if income == 0:
        return 'не указан доход'
    if income <= 80000:
        return 'малообеспеченный'
    if (income >80000) and (income <= med):
        return 'ниже среднего'
    if (income >med) and (income <= 270000):
        return 'выше среднего'
    return 'богатый'

df['class_income'] = df['total_income'].apply(class_income) 
df['class_income'].value_counts() 


выше среднего       8584
ниже среднего       8450
малообеспеченный    2277
богатый             2142
Name: class_income, dtype: int64

**Вывод**

Разбили на категории по уровню дохода, так как в дальнейшем это может пригодиться.  
Разобили на 5 категории, условно: не указан доход, малообеспеченные, ниже среднего, выше среднего и богатые. Учтем, что среднее найдем медианой, а в первой и последней категории должно быть примерно по 10%.

##  Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
# Подсчитываем количество просрочек у заемщиков с детьми и без детей, а также общее количество 
# по этим двум категориям и высчитываем относительную просрочку для каждой категории

total_child_debet = df[df['children'] > 0]['debt'].sum()  
total_child_debet
total_unchild_debet = df[df['children'] == 0]['debt'].sum()
total_unchild_debet
total_child = len(df[df['children'] > 0])
total_child
total_unchild = len(df[df['children'] == 0])
total_unchild
df['children'].value_counts() # Проверим правильность вычислений

part_child_debet = total_child_debet/total_child
part_unchild_debet = total_unchild_debet/total_unchild
part_child_debet, part_unchild_debet # Выводим результат на экран

(0.09208203178052424, 0.07544357700496807)

**Вывод**

Сравним отношение наличия просрочек у заемщиков с детьми и заемщиков без детей.  
В результате мы видим, что 9,2% заемщиков с детьми имеют просрочки и 7,5% заемщиков без детей имеют просрочки. Исходя из этого можно сказать, что заемщики с детьми в среднем на 17% чаще имют просрочку, а значит можно считать, что есть зависимость между наличием детей и возвратом кредита в срок.



- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [16]:
df['family_status'].value_counts() #узнаем количество уникальных значений в столбце семейное положение

# Функция для подсчета показателй
def research(element):
    total_debet = df[df['family_status'] == element]['debt'].sum() # Подсчет количества должников
    total = len(df[df['family_status'] == element]) # Подсчет общего количества по категориям
    part = "{:.2%}".format(total_debet / total) # Подсчет относительной задолжности
    return [element, total_debet, total, part] # Возврат новой строки для категории
    
# Применение функции для каждого статуса
family_status_1 = research('женат / замужем')
family_status_2 = research('гражданский брак')
family_status_3 = research('Не женат / не замужем')
family_status_4 = research('в разводе')
family_status_5 = research('вдовец / вдова')

#Оформление результатов в таблицу
data_family=[family_status_1, family_status_2, family_status_3, family_status_4, family_status_5]
columns = ['family_status','total_debet','total','part']
research_family_status = pd.DataFrame(data=data_family, columns = columns) 
#Сортировка по проценту должников по категориям
research_family_status = research_family_status.sort_values(by='part')

research_family_status


,family_status,total_debet,total,part
4,вдовец / вдова,63,959,6.57%
3,в разводе,85,1195,7.11%
0,женат / замужем,931,12339,7.55%
1,гражданский брак,388,4150,9.35%
2,Не женат / не замужем,274,2810,9.75%


**Вывод**

Пункт похож на предыдущий, но с учетом 5ти вариаций для сравнения, лучше оформить результат таблицей и сравнить. 
Из итоговой таблицы можно увидеть, что семейное положение очень влияет на относительное количество должников, местами разница в полтора раза. Значит есть зависимость между семейным положением и возвратом кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [17]:
# Подготовим таблицу для вывода результатов

data = []
columns = ['class_income','total_debet','total','part']
research_class_income = pd.DataFrame()

class_income = ['не указан доход', 'малообеспеченный', 'ниже среднего', 'выше среднего', 'богатый']

# Цикл для подсчета показателй
for element in class_income:
    total_debet = df[df['class_income'] == element]['debt'].sum() # Подсчет количества должников
    total = len(df[df['class_income'] == element]) # Подсчет общего количества по категориям
    part = "{:.2%}".format(total_debet / total) # Подсчет относительной задолжности
    new_row = [[element, total_debet, total, part]] # Новая строка для категории
    research_class_income = research_class_income.append(new_row) # Добавление новой строки в таблицу с результатами

research_class_income.columns = columns
#Сортировка по проценту должников по категориям
research_class_income = research_class_income.sort_values(by='part')
research_class_income

,class_income,total_debet,total,part
0,богатый,151,2142,7.05%
0,малообеспеченный,174,2277,7.64%
0,выше среднего,678,8584,7.90%
0,ниже среднего,738,8450,8.73%
0,не указан доход,0,0,nan%


**Вывод**

Ранее мы разбили всех заемщиков на 5 категорий по уровню дохода. Сравним относительные задолженности по категориям, по аналогии с предыдущим заданием.

Вывод из данных получается очень интересный и не очевидный:
1. Есть зависимость между уровнем дохода и возвратом кредита в срок, но она не такая яркая, как в предыдущем вопросе.
2. Наибольшее относительное количество долников в средних категориях дохода, далее идет категория с неуказанным доходом.
3. Наименьшее количество должников, как ни странно, в категории богатых, но с небольшим отрывом за не идет категория малообеспеченных.



- Как разные цели кредита влияют на его возврат в срок?

In [18]:
# Подготовим таблицу для вывода результатов

columns = ['purpose_category','total_debet','total','part']
research_purpose_category = pd.DataFrame()

purpose_category = ['свадьба', 'покупка жилья', 'покупка коммерческой недвижимости', 'строительство', 'ремонт','покупка автомобиля', 'образование']

# Цикл для подсчета показателй
for element in purpose_category:
    total_debet = df[df['purpose_category'] == element]['debt'].sum() # Подсчет количества должников
    total = len(df[df['purpose_category'] == element]) # Подсчет общего количества по категориям
    part = "{:.2%}".format(total_debet / total) # Подсчет относительной задолжности
    new_row = [[element, total_debet, total, part]] # Новая строка для категории
    research_purpose_category = research_purpose_category.append(new_row) # Добавление новой строки в таблицу с результатами

research_purpose_category.columns = columns
#Сортировка по проценту должников по категориям
research_purpose_category = research_purpose_category.sort_values(by='part')
research_purpose_category

,purpose_category,total_debet,total,part
0,ремонт,35,607,5.77%
0,покупка жилья,452,6364,7.10%
0,строительство,144,1878,7.67%
0,покупка коммерческой недвижимости,151,1962,7.70%
0,свадьба,186,2323,8.01%
0,образование,370,4013,9.22%
0,покупка автомобиля,403,4306,9.36%


**Вывод**

Мы уже сгруппировали различные цели кредита на 7 групп. Сравним относительные задолженности по этим группам, по аналогии с предыдущим заданием. 
Из итоговой таблицы видно, что цели взятия кредита значительно влияют на возврат в срок. Наименьший процент должников среди целей, связанных с недвижимостью, наибольший для образования и покупки авто. Видимо к операциям с недвижимостью люди подходят более осознанно и подготовленно.

## Общий вывод

По результату работы обнаружено, что все показатели в большей или меньшей степени влияют на возврат кредита в срок. Это означет, что нужно обращать внимание на эти показатели при выдаче кредита и подборе процентной ставки.